In [ ]:
############################################################################
# 日経225 RSI

  #ライブラリーの設定
import pandas as pd
import talib as ta
import numpy as np


def initialize(ctx):
    # 設定
    ctx.logger.debug("initialize() called")
    ctx.configure(
      target="jp.stock.daily",
      channels={          # 利用チャンネル
        "jp.stock": {
          "symbols": [
            "jp.stock.9983", #ファーストリテイリング
            "jp.stock.9984", #ソフトバンク
            "jp.stock.6954", #ファナック
            "jp.stock.9433", #KDDI
            "jp.stock.8028", #ファミリーマート
            "jp.stock.8035", #東京エレクトロン
            "jp.stock.4543", #テルモ
            "jp.stock.6367", #ダイキン
            "jp.stock.6971", #京セラ
            "jp.stock.9735", #セコム
          ],
          "columns": [
            #"open_price_adj",    # 始値(株式分割調整後)
            #"high_price_adj",    # 高値(株式分割調整後)
            #"low_price_adj",     # 安値(株式分割調整後)
            #"close_price",        # 終値
            "close_price_adj",    # 終値(株式分割調整後) 
            #"volume_adj",         # 出来高
            #"txn_volume",         # 売買代金
          ]
        }
      }
    )

    #売買シグナル生成部分
    def _my_RSI(data):

      #各銘柄の終値(株式分割調整後)を取得、欠損データの補完 
      df_close = data["close_price_adj"].fillna(method='ffill')
      
       #RSI設定
      cp = data["close_price_adj"].fillna(method="ffill")
      rsi = pd.DataFrame(data=0,columns=[], index=cp.index)
      for (sym,val) in cp.items():
        rsi[sym] = ta.RSI(cp[sym].values.astype(np.double), timeperiod=14)
      df_rsi_buy=rsi < 25
      df_rsi_sell=rsi > 75
      
       # 売買シグナルの設定
      buy_sig = df_rsi_buy
      sell_sig = df_rsi_sell

      return{
          "rsi": rsi,
          "buy:sig": buy_sig,
          "sell:sig": sell_sig,
        }
        
# シグナル登録
    ctx.regist_signal("my_RSI", _my_RSI)

def handle_signals(ctx, date, current):

    df = current.copy()

    # 買いシグナル
    df_buy = df[df["buy:sig"]]
    if not df_buy.empty:
      for (sym, val) in df_buy.iterrows(): 
        #ctx.logger.info(val)
        sec = ctx.getSecurity(sym)
        msg = "買いシグナル"
        sec.order_target_percent(0.10, comment= msg)

    # 売りシグナル
    df_sell = df[df["sell:sig"]]
    if not df_sell.empty:
      for (sym, val) in df_sell.iterrows(): 
        sec = ctx.getSecurity(sym)
        msg = "売りシグナル"
        sec.order_target_percent(0, comment= msg)